In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import glob

import networkx as nx
from nxviz import CircosPlot
import community

import gudhi as gd
import gudhi.representations
import gudhi.representations.vector_methods


import nilearn
from nilearn import datasets

from pathlib import Path

from nilearn.connectome import ConnectivityMeasure
from nilearn import plotting

In [2]:
from sklearn.preprocessing   import MinMaxScaler
from sklearn.pipeline        import Pipeline
from sklearn.svm             import SVC
from sklearn.ensemble        import RandomForestClassifier
from sklearn.ensemble        import GradientBoostingClassifier

from sklearn.neighbors       import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
aal_labels = nilearn.datasets.fetch_atlas_aal().labels
#ho_labels = nilearn.datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm').labels

### Data view

In [4]:
class DataFMRI:
    
    #input parameters
    timeseries_array = []
    labels = None
    phenotypes_array = []
    connectivity_measure_kind = None
    rips_complex_max_dimension = None
    
    #derived parameters
    matrices = []
    diagrams = []
    simplex_trees = []
    
    def __init__(self, timeseries_array, labels, phenotypes_array,
                 connectivity_measure_kind='correlation', 
                 rips_complex_max_dimension=2):
        self.timeseries_array = timeseries_array
        self.labels = labels
        self.phenotypes_array = phenotypes_array
        self.connectivity_measure_kind = connectivity_measure_kind
        self.rips_complex_max_dimension = rips_complex_max_dimension
        
        # create matrix for each time_series
        self.create_matrices()  
        # Rips complex and persistent diagrams
        self.create_persistence_view()
        
    def create_matrices(self):
        self.matrices=[]
        measure = ConnectivityMeasure(kind=self.connectivity_measure_kind, discard_diagonal=True)
        for i in range(len(self.timeseries_array)):
            matrix = measure.fit_transform([self.timeseries_array[i].values.T])[0]
            self.matrices.append(matrix)
            
        
    def create_persistence_view(self):
        for matrix in self.matrices:
            rips_complex = gudhi.RipsComplex(distance_matrix=1-matrix, max_edge_length=2)
            simplex_tree = rips_complex.create_simplex_tree(max_dimension=self.rips_complex_max_dimension)
            diag=simplex_tree.persistence()
            self.diagrams.append(diag)
            self.simplex_trees.append(simplex_tree)
    
    
    def get_persistence_intervals(self, i, dim):
        return self.simplex_trees[i].persistence_intervals_in_dimension(dim)
        
    def get_persistence_intervals_array(self, dim):
        intervals_array=[]
        for i in range(len(self.timeseries_array)):
            intervals_array.append(self.get_persistence_intervals(i, dim))
        # delete elements with 'inf'
        intervals_array = [intervals_array[i][np.all(np.isfinite(intervals_array[i]), axis=1)] 
                           for i in range(len(intervals_array))]
        return intervals_array

    
    # visualize
    
    def plot_matrix(self, i):
        matrix=self.matrices[i].copy()
        np.fill_diagonal(matrix, 0)
        plotting.plot_matrix(matrix, figure=(10, 8), labels=self.labels, 
                             vmax=1, vmin=matrix.min(), reorder=True)
        
    def plot_persistence_diagram(self, arr_i):
        diagrams_res = self.diagrams[arr_i[0]]
        for i in range(1, len(arr_i)):
            diagrams_res = diagrams_res + self.diagrams[arr_i[i]]
        gudhi.plot_persistence_diagram(diagrams_res, legend=True)
        
    def plot_persistence_barcode(self, arr_i):
        axis = gudhi.plot_persistence_barcode(self.diagrams[arr_i[0]], max_intervals=0, legend=True, alpha=0.3)
        for i in range(1, len(arr_i)):
            gudhi.plot_persistence_barcode(self.diagrams[arr_i[i]], max_intervals=0, legend=True, alpha=0.3, axes=axis)
#         diagrams_res = self.diagrams[arr_i[0]]
#         for i in range(1, len(arr_i)):
#             diagrams_res = diagrams_res + self.diagrams[arr_i[i]]
#         gudhi.plot_persistence_barcode(diagrams_res, max_intervals=0, legend=True)

        
    def plot_persistence_density(self, arr_i):
        diagrams_res = self.diagrams[arr_i[0]]
        for i in range(1, len(arr_i)):
            diagrams_res = diagrams_res + self.diagrams[arr_i[i]]
        gudhi.plot_persistence_barcode(diagrams_res, dimension=1, legend=True)
        

# CNI

In [5]:
#! git clone https://github.com/mdschirmer/2019_CNI_TrainingRelease
#! git clone https://github.com/mdschirmer/2019_CNI_ValidationRelease

In [75]:
paths_aal = []
p = Path("CNI/2019_CNI_TrainingRelease/Training")
for x in p.rglob("timeseries_aal.csv"):
    paths_aal.append(str(x))

timeseries_array = []
for path in paths_aal:
    df = pd.read_csv(path, header=None)
    df['roi'] = aal_labels
    df = df.set_index('roi')
    timeseries_array.append(df)
len(timeseries_array)

200

In [76]:
timeseries_array[0]

,0,1,2,3,4,5,6,7,8,9,...,146,147,148,149,150,151,152,153,154,155
roi,,,,,,,,,,,,,,,,,,,,,
Precentral_L,-969.490,-1017.90,-1447.60,-769.40,265.760,22.7910,-961.820,-611.110,1256.70,2424.50,...,2246.40,2329.600,1695.70,1322.70,1786.100,1941.20,331.47,-2263.00,-3388.60,-2299.10
Precentral_R,-908.290,-991.95,-2016.60,-1191.90,810.150,902.3100,-1059.500,-1777.800,274.22,2527.80,...,2354.40,2665.400,1637.50,108.81,431.200,2041.00,1606.10,-1553.80,-3859.60,-2868.60
Frontal_Sup_L,-530.780,-808.84,-1078.80,114.61,1695.200,1560.3000,-60.992,-930.700,-194.72,670.22,...,732.91,80.974,-777.80,-428.83,1500.700,3075.50,2172.20,-459.38,-1999.30,-1411.40
Frontal_Sup_R,1433.300,1235.90,-752.01,-2083.80,-1291.700,326.3300,770.870,-51.004,-683.09,-323.18,...,1941.30,1319.900,141.83,-591.02,-145.120,785.65,650.62,-699.62,-1419.60,-232.85
Frontal_Sup_Orb_L,246.810,518.42,116.53,-570.03,-737.880,-4.0319,1005.600,1153.600,174.31,-758.48,...,980.40,909.100,-433.87,-1370.60,-665.720,841.23,1434.00,716.71,-182.64,-269.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vermis_6,-2003.400,297.32,1969.30,1774.80,283.070,-930.3200,-1000.500,-458.460,-310.14,-821.31,...,2165.70,988.410,-1594.30,-2590.10,-586.240,2525.00,3651.70,1785.00,-1255.60,-2875.10
Vermis_7,847.280,4221.80,5624.40,2477.00,-2724.200,-4861.8000,-2607.300,21.606,-546.59,-2694.00,...,-2517.60,-3011.000,-2932.10,-2142.50,-19.516,2883.60,4596.40,3706.30,1088.40,-558.76
Vermis_8,-1895.800,-246.68,1047.70,1285.10,97.026,-1260.1000,-1025.400,326.000,630.54,-448.65,...,1509.90,1761.100,-470.12,-2498.90,-993.060,2564.40,3618.20,930.74,-2250.00,-3071.30


In [77]:
phenotypic_training = pd.read_csv('CNI/2019_CNI_TrainingRelease/SupportingInfo/phenotypic_training.csv')
phenotypes_array = phenotypic_training['DX']

In [78]:
paths_aal = []
p = Path("CNI/2019_CNI_ValidationRelease/Validation")
for x in p.rglob("timeseries_aal.csv"):
    paths_aal.append(str(x))

#timeseries_array = []
for path in paths_aal:
    df = pd.read_csv(path, header=None)
    df['roi'] = aal_labels
    df = df.set_index('roi')
    timeseries_array.append(df)
len(timeseries_array)

240

In [79]:
phenotypic_training = pd.read_csv('CNI/2019_CNI_ValidationRelease/SupportingInfo/phenotypic_validation.csv')
phenotypes_array = list(phenotypes_array.values) + list(phenotypic_training['DX'].values)

In [ ]:
data_fMRI_CNI = DataFMRI(timeseries_array, aal_labels, phenotypes_array)

### Learning

In [81]:
pipe = Pipeline([("Separator", gd.representations.DiagramSelector(limit=np.inf, point_type="finite")),
                 ("Scaler",    gd.representations.DiagramScaler(scalers=[([0,1], MinMaxScaler())])),
                 #("TDA",       gd.representations.PersistenceImage()),
                 ("Estimator", GradientBoostingClassifier())])


param =    [{"Scaler__use":         [False, True],
#              "TDA":                 [
#                                      #gd.representations.PersistenceImage(),
#                                      gd.representations.Landscape(),
#                                      gd.representations.Silhouette(),
#                                      gd.representations.TopologicalVector(),
#                                      #gd.representations.vector_methods.BettiCurve()
#                                      ], 
             
             "Estimator":           [GradientBoostingClassifier(),
                                     RandomForestClassifier(),
                                     SVC()]},]

from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')

In [82]:
len(phenotypes_array)

240

##### dim 0

In [14]:
#for i in range(20):
def get_test_train_data(intervals_array, phenotypes_array):
    test_size            = 0.2
    perm                 = np.random.permutation(len(phenotypes_array))
    limit                = int(test_size * len(phenotypes_array))
    test_sub, train_sub  = perm[:limit], perm[limit:]
    train_phenotypes     = np.array(phenotypes_array)[train_sub]
    test_phenotypes      = np.array(phenotypes_array)[test_sub]
    train_intervals      = [intervals_array[i] for i in train_sub]
    test_intervals       = [intervals_array[i] for i in test_sub]

    landscape = gd.representations.Landscape(resolution=200)
    train_intervals_transformed = [landscape.fit_transform([train_intervals[i]])[0] for i in range(len(train_intervals))]
    test_intervals_transformed = [landscape.transform([test_intervals[i]])[0] for i in range(len(test_intervals))]
    # test_intervals_transformed =  landscape.transform(test_intervals)
    #plt.plot(landscape.fit_transform(train_intervals)[0])
    
    return train_intervals_transformed, train_phenotypes, test_intervals_transformed, test_phenotypes

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

def train_net(X_train, y_train, X_test, y_test, n_epochs=2000, lr=0.1):
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)
    loader = DataLoader(list(zip(X_train, y_train)), shuffle=True, batch_size=16)
    
    model = nn.Sequential(
        nn.Linear(1000, 256),
        nn.Dropout(0.01),
        nn.ReLU(),
        nn.Linear(256, 64),
        nn.Dropout(0.01),
        nn.ReLU(),
        nn.Linear(64, 16),
        nn.Dropout(0.01),
        nn.ReLU(),
        nn.Linear(16, 1),
        nn.Sigmoid()
    )
    
    loss_fn = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=0.0001)
    model.train()
    for epoch in range(n_epochs):
        for X_batch, y_batch in loader:
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    model.eval()
    y_pred_train = model(X_train)
    acc = (y_pred_train.round() == y_train).float().mean()
    acc = float(acc)
    print("Model train accuracy: %.2f%%" % (acc*100))
    y_pred_test = model(X_test)
    acc = (y_pred_test.round() == y_test).float().mean()
    acc = float(acc)
    print("Model test accuracy: %.2f%%" % (acc*100))

In [200]:
intervals_array=data_fMRI_CNI.get_persistence_intervals_array(dim=0)

In [201]:
train_net(*get_test_train_data(intervals_array, phenotypes_array), 20000, 0.1)

Model train accuracy: 91.15%
Model test accuracy: 60.42%


##### dim 1

In [226]:
intervals_array=data_fMRI_CNI.get_persistence_intervals_array(dim=1)

In [214]:
train_net(*get_test_train_data(intervals_array, phenotypes_array), 20000, 0.1)

Model train accuracy: 61.46%
Model test accuracy: 60.42%


In [ ]:
train_net(*get_test_train_data(intervals_array, phenotypes_array), 40000, 0.1)

# OASIS

In [5]:
paths_aal = []
count = 0
p = Path("ts_extracted/controls/AAL")
for x in p.rglob("*"):
    if count > 199:
        break
    paths_aal.append(str(x))
    count = count + 1

timeseries_array_control = []
for path in paths_aal:
    df = pd.read_csv(path, header=0)
    
    df = df.drop(df.columns[0], axis=1)
    timeseries_array_control.append(df.T)
len(timeseries_array_control)

200

In [6]:
phenotypes_array = np.repeat('Control', len(timeseries_array_control))

In [7]:
paths_aal = []
p = Path("ts_extracted/patients/AAL")
count = 0
for x in p.rglob("*"):
    if count > 199:
        break
    paths_aal.append(str(x))
    count = count + 1

timeseries_array_patient = []
for path in paths_aal:
    df = pd.read_csv(path, header=0)
    df = df.drop(df.columns[0], axis=1)
    timeseries_array_patient.append(df.T)
len(timeseries_array_patient)

200

In [8]:
phenotypes_array = np.concatenate((phenotypes_array, np.repeat('Patient', len(timeseries_array_patient))))

In [9]:
timeseries_array = timeseries_array_control + timeseries_array_patient
print(len(timeseries_array))
print(len(phenotypes_array))

400
400


In [10]:
timeseries_array[0]

,0,1,2,3,4,5,6,7,8,9,...,154,155,156,157,158,159,160,161,162,163
Precentral_L,-8.893422e-13,-5.928948e-13,-1.734540,-0.092586,1.862979,-0.108709,0.893523,-1.185790e-12,2.964474e-13,-2.075132e-12,...,-8.893422e-13,-8.893422e-13,-1.185790e-12,-1.482237e-12,-1.185790e-12,-0.170545,-0.926252,0.188549,-5.928948e-13,-1.482237e-12
Precentral_R,1.281249e-12,6.406243e-13,-0.900789,-0.900022,0.396735,-1.968380,1.694640,0.000000e+00,1.281249e-12,-3.203122e-13,...,0.000000e+00,3.203122e-13,3.203122e-13,-3.203122e-13,3.203122e-13,-0.446475,-0.482596,0.695155,6.406243e-13,-6.406243e-13
Frontal_Sup_L,-2.852199e-12,-2.592908e-12,0.198797,0.943867,0.047429,1.351179,-1.453833,-3.111490e-12,-2.852199e-12,-4.148653e-12,...,-2.592908e-12,-2.592908e-12,-3.111490e-12,-3.370781e-12,-3.630071e-12,0.408849,-0.391905,-0.575808,-3.111490e-12,-3.111490e-12
Frontal_Sup_R,-1.463180e-12,-1.463180e-12,0.350958,0.626088,0.454146,0.601361,-2.509140,-1.755816e-12,-2.926360e-13,-2.341088e-12,...,-1.463180e-12,-1.755816e-12,-1.463180e-12,-1.463180e-12,-1.755816e-12,0.286025,0.094856,0.000564,-1.463180e-12,-1.755816e-12
Frontal_Sup_Orb_L,6.068847e-13,8.496386e-13,1.943752,-1.007978,1.251268,0.667647,0.363464,2.427539e-13,8.496386e-13,1.213769e-13,...,2.427539e-13,4.855078e-13,1.213769e-13,2.427539e-13,-2.427539e-13,1.295763,1.093217,-1.102977,1.213769e-13,-1.213769e-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vermis_6,6.167809e-13,9.251713e-13,0.619732,0.649327,0.063110,2.158326,-2.674221,6.167809e-13,9.251713e-13,7.195777e-13,...,7.195777e-13,9.251713e-13,5.139841e-13,4.111872e-13,7.195777e-13,-0.675525,-0.236561,0.253745,5.139841e-13,6.167809e-13
Vermis_7,-5.692931e-14,2.277173e-13,-0.357796,1.523732,-2.751463,0.537211,-1.820633,-1.707879e-13,-1.707879e-13,-2.846466e-13,...,0.000000e+00,-1.138586e-13,0.000000e+00,-5.692931e-14,5.692931e-14,0.375002,-0.289486,0.139869,5.692931e-14,-5.692931e-14
Vermis_8,-1.044504e-13,4.178017e-13,-0.952782,1.367280,-0.690302,1.965360,-0.982614,2.089008e-13,2.089008e-13,-4.178017e-13,...,-1.044504e-13,2.089008e-13,-1.044504e-13,-2.089008e-13,-1.044504e-13,-0.074381,-0.447001,0.249443,-2.089008e-13,-1.044504e-13
Vermis_9,-9.822884e-13,-4.365726e-13,-1.029673,1.645610,-0.154229,-1.754125,-2.059352,-6.548589e-13,0.000000e+00,-9.822884e-13,...,-9.822884e-13,-1.091432e-12,-9.822884e-13,-1.309718e-12,-1.309718e-12,-0.649715,-0.646879,0.679008,-1.200575e-12,-1.091432e-12


In [ ]:
data_fMRI_OASIS = DataFMRI(timeseries_array, aal_labels, phenotypes_array)

### Learning

##### dim 0

In [62]:
intervals_array=data_fMRI_OASIS.get_persistence_intervals_array(dim=0)

In [17]:
phenotypes_array = [1 if el=='Patient' else 0 for el in phenotypes_array]

In [64]:
train_net(*get_test_train_data(intervals_array, phenotypes_array), 2000, 0.1)

Model train accuracy: 91.79%
Model test accuracy: 81.67%


##### dim 1

In [12]:
intervals_array=data_fMRI_OASIS.get_persistence_intervals_array(dim=1)

In [21]:
train_net(*get_test_train_data(intervals_array, phenotypes_array), 20000, 0.1)

Model train accuracy: 96.25%
Model test accuracy: 58.75%


In [66]:
train_net(*get_test_train_data(intervals_array, phenotypes_array), 2000, 0.1)

Model train accuracy: 100.00%
Model test accuracy: 76.67%


То же самое, что и для dim 0